In [1]:
import pandas as pd
import numpy as np
from enum import Enum

In [2]:
# Import dictionary
word_freq_data = pd.read_csv('../data/unigram_freq.csv')
word_freq_data.word = word_freq_data.word.astype('str')
word_freq_data['length'] = word_freq_data.word.apply(len)
word_freq_data['used'] = False

In [3]:
word_freq_data[word_freq_data['length']==5].head(500).tail(40)

,word,count,length,used
2854,adobe,27538755,5,False
2863,trees,27378116,5,False
2867,dress,27318959,5,False
2883,refer,27099706,5,False
2890,babes,27016030,5,False
2898,layer,26912270,5,False
2918,spend,26713310,5,False
2928,clock,26624872,5,False
2943,ratio,26457942,5,False
2956,proof,26305059,5,False


In [4]:
# Specify color patterns in python
#     yellow = '\033[93m'
#     green = '\033[92m'
#     red = '\033[91m'
#     blue = '\033[94m'
#     pink = '\033[95m'

color_dict = {
    'g':'\033[92m',
    'y':'\033[93m',
    'r':'\033[91m',
    'k':'\033[0m'
}



In [5]:
class LetterResult(Enum):
    CorrectSpot = 0
    WrongSpot = 1
    NotInTrueWord = 2

In [6]:
def display_guess_result(guess_word, guess_result, colors=['g', 'y', 'r']):
    # Takes in a guess word and guess result (a list of LetterResults)
    # Prints it to console
    text = ''
    for i, letter in enumerate(guess_word):
        text = text + "\033[1m" + color_dict[colors[guess_result[i].value]] + f'{letter}' + color_dict['k']
    print(text)
        

class WordGuess:
    def __init__(self, guess_word, true_word):
        assert len(guess_word)==len(true_word), "Guessed word must be correct length!"
        self.guess_word = guess_word.upper()
        self.true_word = true_word.upper()
        self.score_guess()
    
    def score_guess(self):
        self.guess_result = []
        self.guessed_letters = list(set(list(self.guess_word)))
        self.guessed_letters_right_spot = []
        self.guessed_letters_wrong_spot = []
        self.guessed_letters_not_in_true_word = []
        self.info = []
        for i, letter in enumerate(self.guess_word):
            if letter==self.true_word[i]:
                # Right letter, right place
                self.guess_result.append(LetterResult.CorrectSpot)
                self.guessed_letters_right_spot.append(letter)
                self.info.append((i, letter, LetterResult.CorrectSpot))
            elif letter in self.true_word:
                # Right letter, right place
                self.guess_result.append(LetterResult.WrongSpot)
                self.info.append((i, letter, LetterResult.WrongSpot))
                if letter not in self.guessed_letters_right_spot:
                    self.guessed_letters_wrong_spot.append(letter)
            else:
                self.guess_result.append(LetterResult.NotInTrueWord)
                self.info.append((0, letter, LetterResult.NotInTrueWord))
                self.guessed_letters_not_in_true_word.append(letter)

guesses = ['TRUMPY', 'NIKOLA', 'DANDAZ', 'DANDAN', 'LIENAD', 'DANIEL']

for guess in guesses:
    wordguess = WordGuess(guess, 'Daniel')
    display_guess_result(wordguess.guess_word, wordguess.guess_result)
                

TRUMPY
NIKOLA
DANDAZ
DANDAN
LIENAD
DANIEL


In [7]:
def get_true_word(word_length=5, min_rank=500, random_state=None):
    if random_state:
        true_word = word_freq_data[(word_freq_data.length==word_length) & (word_freq_data.used==False)].iloc[:min_rank].sample(n=1, random_state=random_state).word.iloc[0]
    else:
        true_word = word_freq_data[(word_freq_data.length==word_length) & (word_freq_data.used==False)].iloc[:min_rank].sample(n=1).word.iloc[0]
    
    word_freq_data.loc[word_freq_data.word==true_word, 'used'] = True
    return true_word
    

get_true_word(word_length=5, min_rank=100)

'where'

In [14]:
def get_possible_words(word_length, info, n=5):
    words_right_length = word_freq_data[word_freq_data.length==word_length]
    subset = words_right_length
    for pos, letter, result in list(info):
        if result==LetterResult.CorrectSpot:
            subset = subset[subset.word.apply(lambda x: x.upper()[pos]==letter.upper())]
        elif result==LetterResult.WrongSpot:
            subset = subset[subset.word.apply(lambda x: (x.upper()[pos]!=letter.upper()) and letter.upper() in x.upper())]
        elif result==LetterResult.NotInTrueWord:
            subset = subset[subset.word.apply(lambda x: letter.upper() not in x.upper())]
        else:
            raise Exception('Letter result not recognized!')
    return subset.iloc[:n].word.values

info = {
    (0, 'r', LetterResult.NotInTrueWord),
    (1, 'a', LetterResult.NotInTrueWord),
    (2, 't', LetterResult.NotInTrueWord),
    (3, 'e', LetterResult.NotInTrueWord),
    (4, 's', LetterResult.NotInTrueWord),
    (0, 'm', LetterResult.NotInTrueWord),
    (1, 'u', LetterResult.NotInTrueWord),
    (2, 'c', LetterResult.NotInTrueWord),
    (3, 'h', LetterResult.NotInTrueWord),
    (4, 'o', LetterResult.WrongSpot),
    (0, 'b', LetterResult.NotInTrueWord),
    (1, 'o', LetterResult.WrongSpot),
    (2, 'i', LetterResult.NotInTrueWord),
    (3, 'n', LetterResult.WrongSpot),
    (4, 'k', LetterResult.WrongSpot),
}

get_possible_words(5, info)

array(['known', 'knoll', 'knopf', 'knowl', 'knopp'], dtype=object)

In [11]:
class Game:
    def __init__(self, word_length=5):
        self.word_length = word_length 
        self.guessed_words = []
        self.guesses_used = 0
        self.true_word = get_true_word(word_length).upper()
        self.game_over = False
        self.unguessed_letters = set(list('ABCDEFGHIJKLMNOPQRSTUVWXYZ'))
        self.guessed_letters = set()
        self.guessed_letters_right_spot = set()
        self.guessed_letters_wrong_spot = set()
        self.guessed_letters_not_in_true_word = set()
        self.info = set()
    
    def display_state(self):
        print('Unguessed Letters:', ''.join(sorted(list(self.unguessed_letters))))
        print('Guessed Letters - Right Spot:  ', ''.join(sorted(list(self.guessed_letters_right_spot))))
        print('Guessed Letters - Wrong Spot:  ', ''.join(sorted(list(self.guessed_letters_wrong_spot))))
        print('Guessed Letters - Not in Word: ', ''.join(sorted(list(self.guessed_letters_not_in_true_word))))
    
    def play_game(self, max_guesses=6, display_state=True, suggest_words=False):
        while not self.game_over:
            if display_state:
                self.display_state()
            if suggest_words:
                most_common_possible_words = get_possible_words(self.word_length, self.info)
                print('Top five most common words that fit these clues:', most_common_possible_words)
            guess_word = self.solicit_guess()
            guess = WordGuess(guess_word, self.true_word)
            # Update state based on scored guess
            self.guessed_words.append(guess_word)
            self.guesses_used += 1
            self.unguessed_letters.difference_update(guess.guessed_letters)
            self.guessed_letters.update(guess.guessed_letters)
            self.guessed_letters_right_spot.update(guess.guessed_letters_right_spot)
            self.guessed_letters_wrong_spot.update(guess.guessed_letters_wrong_spot)
            self.guessed_letters_wrong_spot.difference_update(self.guessed_letters_right_spot) # move any letters from wrong spot to right spot if they were subsequently found in the right spot
            self.guessed_letters_not_in_true_word.update(guess.guessed_letters_not_in_true_word)
            self.info.update(guess.info)
            display_guess_result(guess.guess_word, guess.guess_result)
            
            if guess_word == self.true_word:
                self.game_over=True
                print("YOU WIN!")
                print("The word was", self.true_word.upper())
                print("GAME OVER")
            elif self.guesses_used >= max_guesses:
                self.game_over=True
                print("You ran out of turns.")
                print(f'The word was "{self.true_word.upper()}"')
                print("GAME OVER")
            
        
        
    def solicit_guess(self):
        guess_word = input(f'\nGuess a {self.word_length}-letter word: ')
        return guess_word.upper()
    

game = Game(word_length=5)

game.play_game(max_guesses=10, suggest_words=False)

Unguessed Letters: ABCDEFGHIJKLMNOPQRSTUVWXYZ
Guessed Letters - Right Spot:   
Guessed Letters - Wrong Spot:   
Guessed Letters - Not in Word:  


KeyboardInterrupt: Interrupted by user

In [126]:
word_freq_data[word_freq_data.used]

,word,count,length,used
168,links,339926541,5,True
228,terms,277705910,5,True
238,using,269448880,5,True
275,index,242826246,5,True
325,north,217809513,5,True
330,media,216432510,5,True
344,board,212361059,5,True
377,title,196676017,5,True
463,table,165341452,5,True
499,going,155284081,5,True
